In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [3]:
import tensorflow as tf
from transformers import BertTokenizerFast, TFBertForTokenClassification
from datasets import load_dataset
import numpy as np

# Load CoNLL-2003 NER dataset
dataset = load_dataset("conll2003")

# Prepare train and validation sets
train_data = dataset["train"]
valid_data = dataset["validation"]

# Define label map
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        truncation=True,
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [2]:

# Tokenization and data preparation functions

train_data = train_data.map(tokenize_and_align_labels, batched=True)
valid_data = valid_data.map(tokenize_and_align_labels, batched=True)

# Filter out unnecessary columns
train_data = train_data.remove_columns(["tokens", "pos_tags", "chunk_tags", "id", "ner_tags"])
valid_data = valid_data.remove_columns(["tokens", "pos_tags", "chunk_tags", "id", "ner_tags"])

# Convert datasets to TensorFlow format
train_dataset = train_data.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=lambda x: {key: tf.convert_to_tensor([d[key] for d in x]) for key in x[0]}
)

valid_dataset = valid_data.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=lambda x: {key: tf.convert_to_tensor([d[key] for d in x]) for key in x[0]}
)

# Define and compile the model
model = TFBertForTokenClassification.from_pretrained("bert-base-cased", num_labels=num_labels)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, ignore_class=-100)

model.compile(optimizer=optimizer, loss=loss)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=3,
)

# Save the model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

...

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
878/878 [==============================] - 477s 451ms/step - loss: 0.1642 - val_loss: 0.0753
Epoch 2/3
878/878 [==============================] - 399s 454ms/step - loss: 0.0483 - val_loss: 0.0658
Epoch 3/3
878/878 [==============================] - 399s 454ms/step - loss: 0.0300 - val_loss: 0.0634


('bert-ner-conll/tokenizer_config.json',
 'bert-ner-conll/special_tokens_map.json',
 'bert-ner-conll/vocab.txt',
 'bert-ner-conll/added_tokens.json',
 'bert-ner-conll/tokenizer.json')

In [ ]:
model.save_pretrained("bert-ner-conll")
tokenizer.save_pretrained("bert-ner-conll")